# Mike's test case

Begin by making a data file

In [1]:
import numpy as np
f = open('Test.dat','w')
omega = 0.75
for i in range(0,10000):
    T = 0.01*i
    phi = omega*T
    R = np.random.rand(3) + np.cos(phi)
    print(T, R[0], R[1], R[2],file=f)
f.close()

### Now generate the SSA

In [ ]:
import pyEXP
coefs = pyEXP.coefs.Coefs.factory('Test.dat')

# A list with one key.  Keys are integer arrays (or lists).  
# The first column of the data table has the key [0]
keylst = [[0]]
config = {"cosine": (coefs, keylst, [])}

# Window size is half the time series (a good default choice if it's practical)
window = int(len(coefs.Times())/2)

# Keep the first 10 PCs
npc = 10

# An example of make YAML flags (commented out here)
flags = """
Jacobi: true
# Traj: true
"""

# Initialize the MSSA and compute the eigenvalues
ssa = pyEXP.mssa.expMSSA(config, window, npc, flags)
ev = ssa.eigenvalues()

### Plot the eigenvalues

In [ ]:
import matplotlib.pyplot as plt

plt.semilogy(ev, '-o')
plt.xlabel('index')
plt.ylabel('eigenvalue')
plt.show()

### Look at the first four PCs

In [ ]:
pc = ssa.getPC()
nt = pc.shape[0]
for i in range(4):
    plt.plot(coefs.Times()[:nt], pc[:,i], label=str(i))
plt.xlabel('Time')
plt.ylabel('PC')
plt.legend()
plt.show()

### Compute the power spectrum of the PCs

In [ ]:
freq, power = ssa.pcDFT()
for i in range(power.shape[1]):
    plt.plot(freq[0:20], power[0:20,i], label=str(i))
plt.xlabel('Frequency')
plt.ylabel('Power')
plt.legend()
plt.show()

We expect our power at $f=1$ and that's what we find!

### Compute the power spectrum of the reconstruction

In [ ]:
ssa.reconstruct([0, 1])

# Show the available keys . . . 
print("Keylist is:", ssa.getRCkeys())

In [ ]:
recon = ssa.getRC([0, 0])
for i in range(recon.shape[1]):
    plt.plot(coefs.Times(), recon[:,i], label=str(i))
plt.xlabel('time')
plt.ylabel('column(0)')
plt.legend()
plt.show()

Do the reconstruction for all available PCs

In [ ]:
ssa.reconstruct([*range(npc)])

Now plot the power as before.

In [ ]:
key = [0, 0] # First column, first data set

freq, power = ssa.singleDFT(key)

for i in range(2):
    plt.plot(freq[0:40], power[0:40,i], label=str(i))
plt.xlabel('Frequency')
plt.ylabel('Power')
plt.title('This shows the power contribution to the data from PC 0, 1')
plt.legend()
plt.show()

for i in range(2, npc):
    plt.semilogy(freq, power[:,i], label=str(i))
plt.xlabel('Frequency')
plt.ylabel('Power')
plt.title('This shows the power contribution to the data from PC 2 through 9')
plt.legend()
plt.show()

freq, power = ssa.channelDFT()
for i in range(power.shape[1]):
    plt.plot(freq[0:40], power[0:40,i], label=str(i))
plt.xlabel('Frequency')
plt.ylabel('Power')
plt.title('Power in the reconstruction to the data for PC 0, 1')
plt.legend()
plt.show()

This shows the power contribution to the data column 0 from each PC.  It's all PC 0, 1, of course, at the same frequency ($f=1$) as the PC itself.  The remaining power from the noise is spread into the remaining PCs.